In [2]:
import csv
import os
import pandas as pd
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
import numpy as np
import pyarrow

In [3]:
os.getcwd()

'/Users/thomassullivan/projects/GitHub/PyTorchNLPBook/chapters/chapter_3'

In [4]:
df = dd.read_csv('/Users/thomassullivan/projects/GitHub/PyTorchNLPBook/data/spam_test/SMSSpamCollection.tsv',
                delimiter='\t')

In [5]:
df.head()

,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat..."
0,ham,Ok lar... Joking wif u oni...
1,spam,Free entry in 2 a wkly comp to win FA Cup fina...
2,ham,U dun say so early hor... U c already then say...
3,ham,"Nah I don't think he goes to usf, he lives aro..."
4,spam,FreeMsg Hey there darling it's been 3 week's n...


In [6]:
new_columns = ['emailtype', 'text']
df = df.rename(columns=dict(zip(df.columns, new_columns)))

In [7]:
with ProgressBar():
    df.repartition(npartitions=1).to_csv('/Users/thomassullivan/projects/GitHub/PyTorchNLPBook/chapters/chapter_3/part*.csv',
                                        index=False)

[########################################] | 100% Completed |  0.2s


In [8]:
df_cleaned = dd.read_csv('/Users/thomassullivan/projects/GitHub/PyTorchNLPBook/chapters/chapter_3/part*.csv')

In [9]:
df_cleaned.tail()

,emailtype,text
5566,spam,This is the 2nd time we have tried 2 contact u...
5567,ham,Will ü b going to esplanade fr home?
5568,ham,"Pity, * was in mood for that. So...any other s..."
5569,ham,The guy did some bitching but I acted like i'd...
5570,ham,Rofl. Its true to its name


In [27]:
#creating a condition whereby we specify 1 if spam or 2 if not
type_parsed = df_cleaned['emailtype'].apply(lambda x: 1 if x=='spam' else 2, meta=('int64'))
df_derived_stage1 = df_cleaned.drop('emailtype', axis=1)
df_derived_stage2 = df_derived_stage1.assign(Emailtype=type_parsed)
#df_derived_stage3 = df_derived_stage2.rename(columns={'Emailtype':'Email Type'})

In [34]:
text = df_cleaned['text'].apply(lambda x: x, meta='int64')
df_derived_stage3 = df_derived_stage2.drop('text', axis=1) #drop the original x axis 
df_derived_stage4 = df_derived_stage3.assign(TextValues=text)

In [37]:
#name the columns back
df_derived_stage5 = df_derived_stage4.rename(columns={'Emailtype': 'Email Type', 'TextValues': 'Text'})

In [39]:
#cleaned_df = dd.read_csv('part*.csv')
df_derived_stage5.to_csv('')

,Email Type,Text
5566,1,This is the 2nd time we have tried 2 contact u...
5567,2,Will ü b going to esplanade fr home?
5568,2,"Pity, * was in mood for that. So...any other s..."
5569,2,The guy did some bitching but I acted like i'd...
5570,2,Rofl. Its true to its name


In [58]:
cleaned_df.head()

,emailtype,text
0,ham,Ok lar... Joking wif u oni...
1,spam,Free entry in 2 a wkly comp to win FA Cup fina...
2,ham,U dun say so early hor... U c already then say...
3,ham,"Nah I don't think he goes to usf, he lives aro..."
4,spam,FreeMsg Hey there darling it's been 3 week's n...


In [61]:
missing_values = cleaned_df.isnull().sum()
missing_values.compute()

emailtype    0
text         0
dtype: int64

In [63]:
with ProgressBar():
    email_types = cleaned_df['emailtype'].value_counts()
email_types.compute()

ham     6
spam    4
Name: emailtype, dtype: int64